In [1]:
from __future__ import division
import numpy as np
import graphlab
import scipy #to save mat files

### Load Data

In [2]:
HealthyData = graphlab.SFrame.read_csv('../../Datasets/Cbrace/HealthyData.csv',verbose=False)
CBRData = graphlab.SFrame.read_csv('../../Datasets/Cbrace/PatientCBRData.csv',verbose=False)
SCOData = graphlab.SFrame.read_csv('../../Datasets/Cbrace/PatientSCOData.csv',verbose=False)

This non-commercial license of GraphLab Create for academic use is assigned to llonini@ricres.org and will expire on November 11, 2016.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\llonini\AppData\Local\Temp\graphlab_server_1476245664.log.0


In [3]:
HealthyCodes = HealthyData['SubjID'].unique()
HealthyCodes = HealthyCodes.sort()
HealthyCodes = HealthyCodes.to_numpy()
print HealthyCodes
PatientCodes = CBRData['SubjID'].unique()
PatientCodes = PatientCodes.sort()
print PatientCodes

[51 52 53 54 55 56 57 58 59 60 61]
[1L, 2L, 5L, 6L, 8L, 11L, 12L, 13L, 14L, 15L, 16L, 19L, 21L, 24L]


In [4]:
PatientCodes = np.array([1, 2, 5, 6, 8, 11, 14, 15, 16, 19, 24]) #patients with enough sessions

### Use 3 sessions for CBR and SCO data. Remove stairs in CBR data for patients who don't have at least 2 sessions with stairs data

Inspect SCO Data

In [5]:
for s in PatientCodes:
    data = SCOData[SCOData['SubjID']==s]
    for sess in np.unique(data['Session']):
        print s,sess,np.unique(data[data['Session']==sess]['Label'].unique().sort())

1 1 [0 1 2 3 4]
1 2 [0 1 2 3 4]
1 3 [0 1 2 3 4]
1 4 [0 1 2 3 4]
1 5 [0 1 2 3 4]
2 1 [0 1 2 3 4]
2 2 [0 1 2 3 4]
2 3 [0 3 4]
2 4 [0 1 2 3 4]
2 5 [0 3 4]
2 6 [3 4]
2 7 [0 2 3 4]
2 8 [0 1 3 4]
2 9 [0 3 4]
5 1 [0 3 4]
5 2 [0 1 2 3 4]
5 3 [0 1 2 3 4]
6 1 [0 1 2 3 4]
6 2 [0 1 2 3 4]
6 3 [0 1 2 3 4]
6 4 [0 1 2 3 4]
6 5 [0 1 2 3 4]
6 6 [1 2 3 4]
8 1 [0 1 2 3 4]
8 2 [0 3 4]
8 3 [0 3 4]
8 4 [0 3 4]
8 5 [0 3 4]
11 1 [0 3 4]
11 2 [0 3 4]
11 3 [0 3 4]
11 4 [0 1 2 3 4]
11 5 [0 1 2 3 4]
11 6 [0 1 2 3 4]
14 1 [0 1 2 3 4]
14 2 [0 1 2 3 4]
14 3 [0 1 2 3 4]
14 4 [0 1 2 3 4]
15 1 [0 3 4]
15 2 [0 3 4]
15 3 [0 1 2 3 4]
16 1 [0 1 2 3 4]
16 2 [0 3 4]
16 3 [0 1 2 3 4]
16 4 [0 1 2 3 4]
16 5 [0 1 2 3 4]
16 6 [0 3 4]
19 1 [0 3 4]
19 2 [0 1 2 3 4]
19 3 [0 3 4]
24 1 [0 1 2 3 4]
24 2 [0 1 2 3 4]
24 3 [0 1 2 3 4]


In [6]:
print SCOData.num_rows()

28276


Select the 3 sessions to keep for patients who have a variable number of activities in each session
* CBR01: 1,2,3
* CBR05: 1,2,3
* CBR06: 1,2,3
* CBR08: 1,2,3
* CBR14: 1,2,3
* CBR15: 1,2,3
* CBR19: 1,2,3
* CBR24: 1,2,3


* CBR02: 1,2,4
* CBR11: 4,5,6
* CBR16: 1,3,4

In [7]:
tochange =[1,5,6,8,14,15,19,24]
SCODatanew = SCOData.filter_by(tochange,'SubjID')
SCODatanew = SCODatanew.filter_by([1,2,3],'Session')

data = SCOData[SCOData['SubjID']==2].filter_by([1,2,4],'Session')
SCODatanew = SCODatanew.append(data)
data = SCOData[SCOData['SubjID']==11].filter_by([4,5,6],'Session')
SCODatanew = SCODatanew.append(data)
data = SCOData[SCOData['SubjID']==16].filter_by([1,3,4],'Session')
SCODatanew = SCODatanew.append(data)

In [8]:
print SCODatanew.num_rows()

17614


In [9]:
SCOData = SCODatanew

Inspect CBR Data

In [10]:
print CBRData.num_rows()

29175


In [11]:
for s in PatientCodes:
    data = CBRData[CBRData['SubjID']==s]
    for sess in np.unique(data['Session']):
        print s,sess,np.unique(data[data['Session']==sess]['Label'].unique().sort())


1 1 [0 1 2 3 4]
1 2 [0 1 2 3 4]
1 3 [0 1 2 3 4]
1 4 [0 1 2 3 4]
1 5 [0 1 2 3 4]
1 6 [0 1 2 3 4]
2 1 [0 1 2 3 4]
2 2 [0 1 2 3 4]
2 3 [0 1 2 3 4]
2 4 [0 1 2 3 4]
5 1 [0 1 2 3 4]
5 2 [0 1 2 3 4]
5 3 [0 1 2 3 4]
5 4 [0 1 2 3 4]
5 5 [0 1 2 3 4]
6 1 [0 1 3 4]
6 2 [0 1 2 3 4]
6 3 [0 1 2 3 4]
6 4 [0 1 2 3 4]
8 1 [0 3 4]
8 2 [0 3 4]
8 3 [0 3 4]
8 4 [0 1 2 3 4]
8 5 [0 3 4]
11 1 [0 3 4]
11 2 [0 1 2 3 4]
11 3 [0 1 2 3 4]
11 4 [0 1 2 3 4]
14 1 [0 3 4]
14 2 [0 3 4]
14 3 [0 1 2 3 4]
14 4 [0 1 2 3 4]
14 5 [0 1 2 3 4]
15 1 [0 3 4]
15 2 [0 1 2 3 4]
15 3 [0 3 4]
15 4 [0 1 2 3 4]
16 1 [0 1 2 3 4]
16 2 [0 1 2 3 4]
16 3 [0 1 2 3 4]
16 4 [0 1 2 3 4]
19 1 [0 3 4]
19 2 [0 3 4]
19 3 [0 3 4]
19 4 [0 3 4]
24 1 [0 1 2 3 4]
24 2 [0 1 2 3 4]
24 3 [0 1 2 3 4]
24 4 [0 1 2 3 4]
24 5 [0 1 2 3 4]


3 Sessions to keep
* CBR01 1,2,3
* CBR02 1,2,3
* CBR05 1,2,3
* CBR08 1,2,3
* CBR16 1,2,3
* CBR19 1,2,3
* CBR24 1,2,3 (Session 1 might have problems!)


* CBR06 2,3,4
* CBR11 2,3,4
* CBR14 3,4,5
* CBR15 2,3,4

In [12]:
#Rename sessions to 1,2,3
def updatesess(x,c):
    x = x-c
    return x

In [13]:
tochange =[1,2,5,8,16,19,24]
CBRDatanew = CBRData.filter_by(tochange,'SubjID')
CBRDatanew = CBRDatanew.filter_by([1,2,3],'Session')

data = CBRData[((CBRData['SubjID']==6) | (CBRData['SubjID']==11))].filter_by([2,3,4],'Session')
data['Session'] = data['Session'].apply(lambda x: x-1) #rename sessions to 1,2,3
CBRDatanew = CBRDatanew.append(data)

data = CBRData[CBRData['SubjID']==14].filter_by([3,4,5],'Session')
data['Session'] = data['Session'].apply(lambda x: x-2) #rename sessions to 1,2,3
CBRDatanew = CBRDatanew.append(data)

data = CBRData[CBRData['SubjID']==15].filter_by([2,3,4],'Session')
data['Session'] = data['Session'].apply(lambda x: x-1) #rename sessions to 1,2,3
CBRDatanew = CBRDatanew.append(data)

In [14]:
print CBRDatanew.num_rows()

16496


In [15]:
CBRData = CBRDatanew

## Save the sframe

In [16]:
CBRData.save('../../Datasets/Cbrace/CBRData3Sess.csv',format='csv')
SCOData.save('../../Datasets/Cbrace/SCOData3Sess.csv',format='csv')